<a href="https://colab.research.google.com/github/jinkyukim-me/Pytorch_Class_/blob/master/classification_catsdogs_gyu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import auth
auth.authenticate_user()

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torchsummary import summary 
import math

In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [0]:
#parameters
learning_rate = 0.01
training_epochs = 10
batch_size = 10

In [0]:
trans = transforms.Compose([transforms.Resize((200, 200)), transforms.ToTensor()])

train_data = dsets.ImageFolder(root='/content/gdrive/My Drive/project/classification/catsdogs/data/train_data',
                               transform=trans)

In [0]:
#Dataloader
data_loader = DataLoader(dataset = train_data, batch_size = batch_size,
                         shuffle = True, num_workers=2, drop_last=True)

In [0]:
#model CNN
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 192, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )  
        self.layer4 = nn.Sequential(
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer5 = nn.Sequential(
            nn.Linear(9216, 120),
            nn.ReLU(),
            nn.Linear(120, 2),
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = out.view(out.shape[0], -1)
        out = self.layer5(out)
        
        return out


In [0]:
model = CNN().to(device)

In [0]:
test_input = (torch.Tensor(3,3,200,200)).to(device)
test_out = model(test_input)

In [0]:
#loss function
criterion = nn.CrossEntropyLoss().to(device)

In [0]:
#optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [78]:
#training
total_batch = len(data_loader)
print('Learning Started. It will takes some time.')

for epoch in range(training_epochs):
    avg_cost = 0
    
    for num, data in enumerate(data_loader): 
        imgs, labels = data
        imgs = imgs.to(device) #input data
        labels = labels.to(device) #label data
        
        optimizer.zero_grad()
        hypothesis = model(imgs) #model에 input값을 넣으면 가설이 됨
        
        cost = criterion(hypothesis, labels) #가설과 실제 라벨 사이의 차이를 계산하는 loss
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch #cost값을 쌓아줌
        
    print('[Epoch:{}] cost = {}'.format(epoch +1, avg_cost))
print('Learning Finished!')

Learning Started. It will takes some time.
[Epoch:1] cost = 0.6944922804832458
[Epoch:2] cost = 0.6940556764602661
[Epoch:3] cost = 0.6937156915664673
[Epoch:4] cost = 0.6937288641929626
[Epoch:5] cost = 0.6936368942260742
[Epoch:6] cost = 0.6942155361175537
[Epoch:7] cost = 0.6938979625701904
[Epoch:8] cost = 0.6935431957244873
[Epoch:9] cost = 0.6938904523849487
[Epoch:10] cost = 0.6933396458625793
[Epoch:11] cost = 0.6941625475883484
[Epoch:12] cost = 0.6932975053787231
[Epoch:13] cost = 0.6940617561340332
[Epoch:14] cost = 0.6936768889427185
[Epoch:15] cost = 0.6938809752464294
[Epoch:16] cost = 0.6940566301345825
[Epoch:17] cost = 0.6939021348953247
[Epoch:18] cost = 0.693983256816864
[Epoch:19] cost = 0.6939032673835754
[Epoch:20] cost = 0.693553626537323
[Epoch:21] cost = 0.6941351890563965
[Epoch:22] cost = 0.6943682432174683
[Epoch:23] cost = 0.6943042278289795
[Epoch:24] cost = 0.6936665773391724
[Epoch:25] cost = 0.6934695243835449
[Epoch:26] cost = 0.6938038468360901
[Epoch

In [0]:
trans = transforms.Compose([transforms.Resize((200, 200)), transforms.ToTensor()])

test_data = dsets.ImageFolder(root='/content/gdrive/My Drive/project/classification/catsdogs/data/test_data',
                               transform=trans)

In [0]:
test_set = DataLoader(dataset = test_data, batch_size = len(test_data))

In [82]:
with torch.no_grad():
    for num, data in enumerate(test_set):
        imgs, label = data
        imgs = imgs.to(device)
        label = label.to(device)
        
        prediction = model(imgs)
        
        correct_prediction = torch.argmax(prediction, 1) == label
        
        accuracy = correct_prediction.float().mean()
        print('Accuracy:', accuracy.item())


RuntimeError: ignored

In [28]:
summary(model, input_size = (3,200,200))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 100, 100]           1,792
              ReLU-2         [-1, 64, 100, 100]               0
         MaxPool2d-3           [-1, 64, 50, 50]               0
            Conv2d-4          [-1, 192, 50, 50]         110,784
              ReLU-5          [-1, 192, 50, 50]               0
         MaxPool2d-6          [-1, 192, 25, 25]               0
            Conv2d-7          [-1, 384, 25, 25]         663,936
              ReLU-8          [-1, 384, 25, 25]               0
         MaxPool2d-9          [-1, 384, 12, 12]               0
           Conv2d-10          [-1, 256, 12, 12]         884,992
             ReLU-11          [-1, 256, 12, 12]               0
        MaxPool2d-12            [-1, 256, 6, 6]               0
           Linear-13                  [-1, 120]       1,106,040
             ReLU-14                  [